In [1]:
# ===============================
# STEP 1: Install Dependencies
# ===============================
!pip install transformers torch Pillow requests

# ===============================
# STEP 2: Load BLIP Model
# ===============================
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
import IPython.display as display

# Load model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# ===============================
# STEP 3: Upload an Image
# ===============================
from google.colab import files
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
image = Image.open(image_path).convert('RGB')
display.display(image)

# ===============================
# STEP 4: Generate Caption
# ===============================
inputs = processor(image, return_tensors="pt")
outputs = model.generate(**inputs)
caption = processor.decode(outputs[0], skip_special_tokens=True)

if not caption.strip():
    caption = "This is a default caption."

print("Generated Caption:", caption)

# ===============================
# STEP 5: Convert Caption to Voice using ElevenLabs
# ===============================

# Replace this with your actual ElevenLabs API key
api_key = "sk_4403d175b2b3d7c9b909aaa6e89a74492c2fb3995f9b5770"

voice_id = "EXAVITQu4vr4xnSDxMaL"  # Default: Rachel

url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
headers = {
    "xi-api-key": api_key,
    "Content-Type": "application/json"
}
data = {
    "text": caption,
    "model_id": "eleven_monolingual_v1",
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.75
    }
}

response = requests.post(url, headers=headers, json=data)

# ===============================
# STEP 6: Play Audio in Colab
# ===============================
from IPython.display import Audio

if response.status_code == 200:
    with open("voice_caption.mp3", "wb") as f:
        f.write(response.content)
    print("🔊 Voice generated successfully!")
    display.display(Audio("voice_caption.mp3", autoplay=True))
else:
    print("❌ Voice generation failed!")
    print(response.text)  # To see the exact error

Output hidden; open in https://colab.research.google.com to view.